# Задача

Делаем анализ, который обсуждали.

*Исходник данных*: продажи за 2 года.

1. На 2016.12.31 получаем инфу по кол-ву заказов на каждого юзера. Подсчитываем сколько пользователей попало в группу с 1 заказом, 2, 3 и т.д.
2. За январь 2017 смотрим, какое кол-во юзеров из п.1 купило в каждой группе и на какую сумму за месяц.(скрипты вам должны помочь)
3. Подсчитываем вероятность перехода в покупающего по каждой группе и ср траты за январь в группе (как на занятии).
4. В идеале сделать тоже самое на февраль, март, и до конца 17 года (чтобы оценить сезонность и рост трат за месяц)

# Подготовка

## Подключение библиотек

In [42]:
import pymysql.cursors
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import getpass

## Подключение к базе данных

In [43]:
# password_mysql = getpass.getpass(prompt="Введите пароль от MySQL-сервера: ")
password_mysql = "pass123"
mydb = pymysql.connect(host="127.0.0.1",
                             user='root',
                             password=password_mysql, 
                             db='db_analyt_loc',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
print ("connect successful!!")

connect successful!!


In [44]:
def show_tables():
    return pd.read_sql("""SHOW TABLES FROM db_analyt_loc;""", con=mydb)
show_tables()

,Tables_in_db_analyt_loc
0,orders_20190822
1,orders_all
2,time_user_b


### 1. На 2016.12.31 получаем инфу по кол-ву заказов на каждого юзера. Подсчитываем сколько пользователей попало в группу с 1 заказом, 2, 3 и т.д.

In [30]:
%%time
# Второй раз выполнять не надо
mydb.cursor().execute(
    """
    CREATE TABLE time_user_b
        SELECT
          user_id,
          COUNT(id_o) n,
          DATE('2017-01-01') m
        FROM orders_20190822 o
        WHERE DATE(o_date) < DATE('2017-01-01')
        GROUP BY user_id;
    """)
mydb.commit()

months = ['02','03','04','05','06','07','08','09','10','11','12']
for month in months:
    mydb.cursor().execute(
        f"""
        INSERT INTO time_user_b
            SELECT
            user_id,
            COUNT(id_o) n,
            DATE('2017-{month}-01')
            FROM orders_20190822 o
            WHERE DATE(o_date) < DATE('2017-{month}-01')
            GROUP BY user_id;
          """)
    mydb.commit()

Wall time: 2min 30s


In [45]:
show_tables()

,Tables_in_db_analyt_loc
0,orders_20190822
1,orders_all
2,time_user_b


In [46]:
mydb.cursor().execute(
    """
    CREATE TABLE time_user_ym_s
        SELECT
          user_id,
          DATE_FORMAT(o_date, '%y%m') ym,
          SUM(price) s
        FROM orders_20190822 o
        GROUP BY user_id, DATE_FORMAT(o_date, '%y%m');
    """)
mydb.commit()
show_tables()

,Tables_in_db_analyt_loc
0,orders_20190822
1,orders_all
2,time_user_b
3,time_user_ym_s


### 2. За январь 2017 смотрим, какое кол-во юзеров из п.1 купило в каждой группе и на какую сумму за месяц.(скрипты вам должны помочь)

### 3. Подсчитываем вероятность перехода в покупающего по каждой группе и ср траты за январь в группе (как на занятии).

### 4. В идеале сделать тоже самое на февраль, март, и до конца 17 года (чтобы оценить сезонность и рост трат за месяц)

In [7]:
%%time
mydb.cursor().execute(
    """
    CREATE TABLE time_user_ym_s
    SELECT
      user_id,
      DATE_FORMAT(o_date, '%y%m') AS ym,
      SUM(price) AS s
    FROM orders_20190822 o
    GROUP BY user_id, DATE_FORMAT(o_date, '%y%m');
    """
)
mydb.commit()

Wall time: 34.4 s


In [35]:
pd.read_sql(f"SELECT * FROM time_user_ym_s", con=mydb)

,user_id,ym,s
0,337544,1601,539.0
1,171642,1601,3584.7
2,260596,1601,55.3
3,1105609,1601,752.5
4,982696,1601,4410.0
...,...,...,...
1524620,1156728,1712,1172.5
1524621,5806002,1712,825.3
1524622,5919142,1712,4934.3
1524623,5919156,1712,5019.7


# Закрытие сессии

In [18]:
mydb.close()